# <div style="text-align: center; color: #1a5276;">Custom Models</div>

## <font color='blue'>  Table of Contents </font>

1. [Introduction](#1)
2. [Setup](#2)
3. [Model](#3)
4. [References](#references)

<a name="1"></a>
## <font color='blue'> 1. Introduction </font>

This notebook demonstrates how to build a custom model in PyTorch, with a focus on creating a Wide & Deep Network. The Wide & Deep architecture combines linear models (for memorization) with deep neural networks (for generalization), making it particularly effective for recommendation systems, tabular data, and structured data problems.

<a name="2"></a>
## <font color='blue'> 2. Setup </font>

In [1]:
#!pip install torchview

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchview import draw_graph

In [5]:
torch.manual_seed(42)

<a name="3"></a>
## <font color='blue'> 3. Model </font>

We will build the following model:
    
<img src="images/wide_deep.png"/>

In [8]:
class WideAndDeepModel(nn.Module):
    """
    A custom PyTorch model implementing the Wide & Deep architecture.

    This model combines:
    - A wide branch (linear model) for memorization.
    - A deep branch (multi-layer network) for generalization.
    
    Parameters:
    -----------
    num_wide_features : int
        Number of input features for the wide branch.
    num_deep_features : int
        Number of input features for the deep branch.
    units : int, optional (default=30)
        Number of units in the hidden layers of the deep branch.

    Returns:
    --------
    main_output : torch.Tensor
        The primary prediction output (e.g., for classification).
    aux_output : torch.Tensor
        An auxiliary output, often used for additional supervision or regularization.
    """

    def __init__(self, num_wide_features, num_deep_features, units=30):
        super(WideAndDeepModel, self).__init__()
        
        # Wide branch - linear layer for wide features
        self.wide = nn.Linear(num_wide_features, 1)
        
        # Deep branch - two hidden layers for deep features
        self.hidden1 = nn.Linear(num_deep_features, units)
        self.hidden2 = nn.Linear(units, units)
        
        # Main output - combines wide and deep features
        self.main_output = nn.Linear(num_wide_features + units, 1)
        
        # Auxiliary output - based on deep features alone
        self.aux_output = nn.Linear(units, 1)

    def forward(self, inputs):
        """
        Forward pass of the model.

        Parameters:
        -----------
        inputs : tuple of torch.Tensor
            Contains two tensors:
            - input_wide: Tensor for the wide branch features.
            - input_deep: Tensor for the deep branch features.

        Returns:
        --------
        tuple:
            - main_output (torch.Tensor): Primary prediction output.
            - aux_output (torch.Tensor): Auxiliary prediction output.
        """
        input_wide, input_deep = inputs
        
        # Deep branch forward pass with ReLU activation
        hidden1 = F.relu(self.hidden1(input_deep))
        hidden2 = F.relu(self.hidden2(hidden1))
        
        # Concatenate wide and deep outputs
        concat = torch.cat([input_wide, hidden2], dim=1)
        
        # Final outputs with sigmoid activation
        main_output = torch.sigmoid(self.main_output(concat))
        aux_output = torch.sigmoid(self.aux_output(hidden2))
        
        return main_output, aux_output


Let's try our model:

In [10]:
# Example usage
input_wide = torch.randn(5, 5)  # Wide features
input_deep = torch.randn(5, 10)  # Deep features
model = WideAndDeepModel(num_wide_features=5, num_deep_features=10)
main_output, aux_output = model((input_wide, input_deep))

print(f"Main Output: {main_output}")
print(f"Aux Output: {aux_output}")

Main Output: tensor([[0.5596],
        [0.5176],
        [0.5808],
        [0.5815],
        [0.5987]], grad_fn=<SigmoidBackward0>)
Aux Output: tensor([[0.5146],
        [0.5082],
        [0.5182],
        [0.4988],
        [0.4981]], grad_fn=<SigmoidBackward0>)


In [15]:
# Inline plot of the model
model_graph = draw_graph(model, input_data=((input_wide, input_deep),), expand_nested=True)
model_graph.visual_graph.render("images/wide_and_deep_model", format="png")

'images/wide_and_deep_model.png'

<img src="images/wide_and_deep_model.png"/>

Let's implement a couple of unit tests:

In [16]:
# Unit tests
def test_model_output_shape():
    model = WideAndDeepModel(num_wide_features=5, num_deep_features=10)
    main_output, aux_output = model((input_wide, input_deep))
    assert main_output.shape == (5, 1), f"Expected shape (5, 1), got {main_output.shape}"
    assert aux_output.shape == (5, 1), f"Expected shape (5, 1), got {aux_output.shape}"

def test_forward_pass():
    model = WideAndDeepModel(num_wide_features=5, num_deep_features=10)
    main_output, aux_output = model((input_wide, input_deep))
    assert torch.all((main_output >= 0) & (main_output <= 1)), "Main output values should be in range [0, 1]"
    assert torch.all((aux_output >= 0) & (aux_output <= 1)), "Aux output values should be in range [0, 1]"

test_model_output_shape()
test_forward_pass()
print("All tests passed!")

All tests passed!


<a name="references"></a>
## <font color='blue'> References </font>

[PyTorch Documentation](https://pytorch.org/docs/stable/index.html)